-  DTM이나 TF-IDF의 치명적 단점
    - 문서의 의미나 주제 등을 알 수 없다.  

# Latent Semantic Analysis (LSA)
- 잠재 의미 분석
- 단어들 사이의 관계를 찾는 기술
- 단어-단어, 문서-문서, 단어-문서 사이의 의미적 유사성 점수를 찾을 수 있다.

In [6]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os

In [4]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
dir_path = '../../data/topic_modelling/'
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

csv_filename = os.path.join(dir_path, 'abcnews-date-text.csv')

urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", 
                           filename=csv_filename)

('../../data/topic_modelling/abcnews-date-text.csv',
 <http.client.HTTPMessage at 0x1b338f6b370>)

In [14]:
data = pd.read_csv(csv_filename, on_bad_lines='skip')
data.shape

(1082168, 2)

In [15]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [17]:
# publish_data는 불필요 - 제거
text = data[['headline_text']].copy()
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [19]:
# 데이터 중복 확인
text.nunique() # 중복을 제외하고 유일한 시퀀스를 가지는 샘플의 개수를 출력

headline_text    1054983
dtype: int64

1,082,168개 중 1,054,983개의 중복없는 샘플 존재

In [20]:
text.drop_duplicates(inplace=True) # 중복 샘플 제거
text.reset_index(drop=True, inplace=True)
text.shape

(1054983, 1)

In [21]:
# NLTK 토크나이저를 이용해서 토큰화
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

# 불용어 제거
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])

text.head()

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


In [22]:
# 단어 정규화. 3인칭 단수 표현 -> 1인칭 변환, 과거형 동사 -> 현재형 동사 등을 수행한다.
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 길이가 1 ~ 2인 단어는 제거.
text = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 2])
print(text[:5])

0     [aba, decide, community, broadcast, licence]
1    [act, fire, witness, must, aware, defamation]
2       [call, infrastructure, protection, summit]
3            [air, staff, aust, strike, pay, rise]
4    [air, strike, affect, australian, travellers]
Name: headline_text, dtype: object


In [25]:
# 역토큰화 (토큰화 작업을 역으로 수행)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(text[i])
    detokenized_doc.append(t)

train_data = detokenized_doc
train_data[:5]

['aba decide community broadcast licence',
 'act fire witness must aware defamation',
 'call infrastructure protection summit',
 'air staff aust strike pay rise',
 'air strike affect australian travellers']

In [38]:
train_data[:10]

['aba decide community broadcast licence',
 'act fire witness must aware defamation',
 'call infrastructure protection summit',
 'air staff aust strike pay rise',
 'air strike affect australian travellers',
 'ambitious olsson win triple jump',
 'antic delight record break barca',
 'aussie qualifier stosur waste four memphis match',
 'aust address security council iraq',
 'australia lock war timetable opp']

In [28]:
# 상위 5000개의 단어만 사용 DTM 생성
c_vectorizer = CountVectorizer(stop_words='english', max_features = 5000)
document_term_matrix = c_vectorizer.fit_transform(train_data)

In [29]:
print('행렬의 크기 :',document_term_matrix.shape)

행렬의 크기 : (1054983, 5000)


In [31]:
from sklearn.decomposition import TruncatedSVD

n_topics = 10
lsa_model = TruncatedSVD(n_components = n_topics)
lsa_model.fit_transform(document_term_matrix)

array([[ 1.20227398e-02, -3.62725922e-03,  1.82401971e-02, ...,
         3.08333149e-03, -6.98486076e-05,  1.56272878e-02],
       [ 2.90369857e-02, -1.08904834e-02,  1.81885024e-02, ...,
         2.78291139e-04, -1.00586889e-02, -8.79072807e-03],
       [ 5.02394018e-03, -1.98994571e-03,  9.74619614e-03, ...,
        -2.99896551e-03,  4.24389334e-04,  2.31858553e-03],
       ...,
       [ 2.96586519e-02,  4.31795957e-03,  2.50247289e-02, ...,
         3.29679061e-02,  1.72278636e-02,  1.72989451e-02],
       [ 6.12485357e-02, -5.51884264e-03,  1.37382533e-01, ...,
         8.94902147e-01,  8.69376281e-01, -2.01784473e-01],
       [ 7.13744537e-02,  2.85159997e-02,  3.47772185e-04, ...,
         1.13124255e-02, -3.68131783e-02, -1.84006723e-02]])

In [32]:
print(lsa_model.components_.shape)

(10, 5000)


In [35]:
lsa_model.components_.

array([0.00208478, 0.00059315, 0.00034665, ..., 0.00165353, 0.00222405,
       0.00081975])

In [37]:
c_vectorizer.get_feature_names_out()

array(['100', '1000', '10000', ..., 'zimbabwe', 'zone', 'zoo'],
      dtype=object)

In [33]:
terms = c_vectorizer.get_feature_names_out() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lsa_model.components_, terms)

Topic 1: [('police', 0.74691), ('man', 0.45382), ('charge', 0.211), ('new', 0.14082), ('court', 0.11143)]
Topic 2: [('man', 0.69458), ('charge', 0.30061), ('court', 0.16839), ('face', 0.11415), ('murder', 0.10551)]
Topic 3: [('new', 0.83792), ('plan', 0.23665), ('say', 0.18261), ('govt', 0.10994), ('council', 0.10936)]
Topic 4: [('say', 0.73845), ('plan', 0.35795), ('govt', 0.16999), ('council', 0.12928), ('urge', 0.07197)]
Topic 5: [('plan', 0.73231), ('council', 0.18182), ('govt', 0.13824), ('urge', 0.08429), ('water', 0.07046)]
Topic 6: [('govt', 0.51488), ('court', 0.27498), ('urge', 0.27293), ('fund', 0.22205), ('face', 0.18088)]
Topic 7: [('charge', 0.52126), ('court', 0.45104), ('face', 0.34014), ('plan', 0.12611), ('murder', 0.12253)]
Topic 8: [('win', 0.63721), ('court', 0.35255), ('kill', 0.14392), ('crash', 0.11369), ('australia', 0.09134)]
Topic 9: [('win', 0.54128), ('charge', 0.50856), ('world', 0.09931), ('cup', 0.0992), ('australia', 0.09908)]
Topic 10: [('council', 0.8

# Latent Dirichlet Allocation (LDA)
- DTM 또는 TM-IDF를 입력으로 받는다.

In [39]:
# TF-IDF 행렬 활용

# 상위 5,000개의 단어만 사용
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tf_idf_matrix = tfidf_vectorizer.fit_transform(train_data)

# TF-IDF 행렬의 크기를 확인해봅시다.
print('행렬의 크기 :', tf_idf_matrix.shape)

행렬의 크기 : (1054983, 5000)


In [40]:
# scikit-learn LDA 모델
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_model.fit_transform(tf_idf_matrix)

array([[0.0335099 , 0.0335099 , 0.0335099 , ..., 0.1702395 , 0.0335099 ,
        0.03351907],
       [0.03365628, 0.03365628, 0.03365628, ..., 0.03365628, 0.03365628,
        0.03365628],
       [0.25184095, 0.0366096 , 0.0366096 , ..., 0.45528225, 0.0366096 ,
        0.0366096 ],
       ...,
       [0.26688724, 0.02914502, 0.14077174, ..., 0.02914502, 0.02914502,
        0.02914502],
       [0.02637829, 0.0263834 , 0.11651895, ..., 0.39092402, 0.02637829,
        0.02637829],
       [0.03376055, 0.03376055, 0.03376055, ..., 0.40866263, 0.21169867,
        0.03376966]])

In [41]:
print(lda_model.components_.shape)

(10, 5000)


In [42]:
get_topics(lda_model.components_, terms)

Topic 1: [('home', 4048.78778), ('hit', 3572.60152), ('market', 3141.55263), ('ban', 2995.63559), ('rise', 2909.02651)]
Topic 2: [('australia', 6721.04185), ('perth', 4551.37242), ('kill', 3977.09633), ('year', 3925.53959), ('say', 3395.39353)]
Topic 3: [('say', 7349.92516), ('court', 5250.21134), ('open', 3770.65617), ('state', 3656.60868), ('face', 3611.67807)]
Topic 4: [('man', 6521.51056), ('police', 6333.33358), ('charge', 5948.07591), ('queensland', 5552.61778), ('murder', 4677.1475)]
Topic 5: [('melbourne', 5298.43132), ('school', 3966.56666), ('report', 3792.92855), ('rural', 3521.89517), ('warn', 3379.34835)]
Topic 6: [('australian', 7674.39181), ('world', 4536.14226), ('country', 4168.83782), ('day', 3852.20674), ('crash', 3794.11334)]
Topic 7: [('election', 5416.23599), ('adelaide', 4868.15748), ('south', 4852.30258), ('house', 4481.58381), ('make', 3773.26989)]
Topic 8: [('canberra', 4323.53542), ('2016', 3962.69218), ('win', 3874.8274), ('qld', 3229.28135), ('price', 2762.

# 형태소 분석기 - 한글

In [2]:
en_text = "The dog ran back to the corner near the spare bedrooms"
print(en_text.split())

['The', 'dog', 'ran', 'back', 'to', 'the', 'corner', 'near', 'the', 'spare', 'bedrooms']


In [3]:
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사 왔어"
print(kor_text.split())

['사과의', '놀라운', '효능이라는', '글을', '봤어.', '그래서', '오늘', '사과를', '먹으려고', '했는데', '사과가', '썩어서', '슈퍼에', '가서', '사과랑', '오렌지', '사', '왔어']


사과 - 4번 등장 (사과의, 사과를, 사과가, 사과랑)

In [4]:
from konlpy.tag import Okt

tokenizer = Okt()
print(tokenizer.morphs(kor_text))

['사과', '의', '놀라운', '효능', '이라는', '글', '을', '봤어', '.', '그래서', '오늘', '사과', '를', '먹으려고', '했는데', '사과', '가', '썩어서', '슈퍼', '에', '가서', '사과', '랑', '오렌지', '사', '왔어']


In [5]:
print(tokenizer.morphs('모두의연구소에서 자연어 처리를 공부하는 건 정말 즐거워'))

['모두', '의', '연구소', '에서', '자연어', '처리', '를', '공부', '하는', '건', '정말', '즐거워']


## Soynlp

In [8]:
import urllib.request
import os

dir_path = '../../data/topic_modelling/'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

txt_filename = os.path.join(dir_path, '2016-10-20.txt')

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt",\
                            filename=txt_filename)

('../../data/topic_modelling/2016-10-20.txt',
 <http.client.HTTPMessage at 0x2021e1956c0>)

In [13]:
from soynlp import DoublespaceLineCorpus

# 말뭉치에 대해서 다수의 문서로 분리
corpus = DoublespaceLineCorpus(txt_filename)
len(corpus)

30091

In [14]:
i = 0
for document in corpus:
  if len(document) > 0:
    print(document)
    i = i+1
  if i == 3:
    break

19  1990  52 1 22
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에

In [17]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 1.005 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


### soynlp의 응집 확률(cohesion probability)

In [19]:
word_score_table["반포한"].cohesion_forward

0.08838002913645132

In [20]:
word_score_table["반포한강"].cohesion_forward

0.19841268168224552

In [21]:
word_score_table["반포한강공"].cohesion_forward

0.2972877884078849

In [22]:
word_score_table["반포한강공원"].cohesion_forward

0.37891487632839754

In [23]:
word_score_table["반포한강공원에"].cohesion_forward

0.33492963377557666

### soynlp의 브랜칭 엔트로피(branching entropy)

In [25]:
word_score_table["디스"].right_branching_entropy

1.6371694761537934

In [26]:
word_score_table["디스플"].right_branching_entropy

-0.0

In [27]:
word_score_table["디스플레"].right_branching_entropy

-0.0

In [28]:
word_score_table["디스플레이"].right_branching_entropy

3.1400392861792916

### soynlp의 LTokenizer

In [32]:
# 띄어쓰기 단위로 잘 나뉜 문장
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

In [33]:
# 띄어쓰기가 되어있지 않은 문장
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

# 용어 정리

**DTM (Document-Term Matrix)**: 문서에서 단어의 등장 빈도를 행렬로 표현한 것.

**TF-IDF (Term Frequency-Inverse Document Frequency)**: 자주 등장하지만 전체 문서에서 흔하지 않은 단어에 더 높은 가중치를 주는 방식.

**LSA (Latent Semantic Analysis)**: DTM이나 TF-IDF 행렬을 차원 축소해 숨겨진 의미(잠재 의미)를 파악하는 기법.

**LDA (Latent Dirichlet Allocation)**: 문서들이 여러 주제(topic)로 구성된다고 보고, 각 문서에 어떤 주제가 포함됐는지 확률적으로 추정하는 기법.

**soynlp** 
- 한국어 자연어 처리를 위해 개발된 파이썬 기반 오픈소스 라이브러리로, 특히 비지도 학습 기반의 형태소 분석 및 단어 추출에 강점.

- 형태소 사전 없이 **통계적 방법(예: L-R, cohesion, branching entropy 등)**으로 새로운 단어나 신조어를 자동으로 추출할 수 있어, 미등록 단어 문제 해결에 유용.

- 또한, 단어 점수 기반 토큰화(tokenizer) 기능도 제공해, 띄어쓰기 오류가 많은 텍스트에도 효과적으로 대응할 수 있음.